# Generate noise series for the production cost of generators

This notebook generates noise series for all 770 dispatchable generators of the European model.

In [1]:
import os, sys
sys.path.append('../TimeSeries/')

import time_series as ts

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Get a list of gens

Import the network file:

In [3]:
with open('../models/europe.json') as f:
    network = json.load(f)

Count the number of generators:

In [4]:
n_gens_all = len(network['gen'])
n_gens_all

815

Subtract non-dispatchable generators:

In [5]:
n_nuc = len(pd.read_csv('data/nuclear_2016.csv'))
n_nuc

44

In [6]:
n_gens = n_gens_all - n_nuc
n_gens

771

### Generate noise series

To be relevant, the noise on the generation cost must be of the same order of magnitude as the line cost computed in the model, which is directly related to the power flowing on the line.

We choose therefore a noise with a standard deviation of 1 MW (1% of a typical line load of 100 MW).

In [7]:
noise_time_series = ts.generate_noise(count=n_gens)

In [8]:
noise_time_series.mean(), noise_time_series.std(axis=1).mean()

(-5.426638784618022e-16, 0.9910005138116696)

Pack everything into a dataframe, with time steps as columns and generators as rows:

In [9]:
df = pd.DataFrame(noise_time_series)
df.index.name = 'id'
df

,0,1,2,3,4,5,6,7,8,9,...,8726,8727,8728,8729,8730,8731,8732,8733,8734,8735
id,,,,,,,,,,,,,,,,,,,,,
0,1.587772,0.848739,0.211134,0.559697,0.640619,2.875499,0.057433,-0.043240,3.158638,1.906316,...,0.508718,1.755275,-0.391129,-1.184850,0.236555,0.768243,0.266537,-0.769761,-0.493886,-0.056240
1,0.993581,0.610868,0.768369,0.948534,0.437705,0.675325,0.780884,0.220308,0.440593,0.901260,...,1.727527,1.908813,1.921934,1.729796,1.177036,1.895706,1.682864,0.408121,1.255829,1.657474
2,-1.819069,-1.374773,-1.274076,-1.307025,-1.102045,-0.867051,-0.915302,-1.112280,-0.748028,-0.547975,...,-1.623955,-0.329383,-0.716157,-1.114035,-0.669882,-0.953574,-0.662542,-1.196687,-0.774894,-0.753788
3,-0.704078,-0.616667,-0.814239,-1.530845,-1.349924,-1.091546,-0.432309,-0.566426,-0.713744,0.354408,...,1.458252,0.080353,-0.781160,0.025421,-0.211286,0.445272,1.443770,1.370113,1.137001,0.344171
4,-1.567261,-0.441336,-0.567468,-1.785128,-1.909993,-0.806943,-0.548662,-0.942096,-1.820165,-1.940507,...,0.735916,0.957282,0.596986,-0.084556,0.264438,-0.266003,-0.948843,-0.318836,-1.201917,-2.426599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,0.464441,0.669763,1.202785,1.058231,0.934235,1.271015,1.397469,1.321312,1.445583,1.546567,...,1.039575,0.574697,1.254652,1.963026,1.351931,0.832973,0.728592,1.256221,0.944892,0.376371
767,-0.217927,0.113563,0.093070,0.117657,0.549181,0.029884,0.299867,0.007868,-0.209018,-0.362873,...,-0.447867,-0.247390,1.068820,-0.397097,0.741579,0.657132,-0.375943,1.060021,-0.104931,-0.763680
768,-2.063161,-2.175277,-0.929543,-1.169798,-0.813126,-2.363415,-1.850062,-1.665087,-2.669921,-2.073052,...,-0.088977,-1.219719,0.035286,-0.251101,-0.361223,-1.390771,-2.053888,0.077829,-0.317766,-0.708482


Export to CSV format:

In [14]:
df.to_csv('data/cost_noise.csv', mode='x', index=False)

As with the loads, the whole procedure can be iterated:

In [10]:
for year in range(2016, 2021):
    n_nuc = len(pd.read_csv('data/nuclear_%d.csv' % year))
    n_gens = n_gens_all - n_nuc
    for i in range(1, 5):
        df = pd.DataFrame(ts.generate_noise(count=n_gens))
        df.index.name = 'id'
        filename = "data/cost_noise_%d_%d.csv" % (year, i)
        print("Writing to file '%s'..." % filename, end='')
        df.to_csv(filename, mode='x', index=False)
        print("ok")

Writing to file 'data/cost_noise_2016_1.csv'...ok
Writing to file 'data/cost_noise_2016_2.csv'...ok
Writing to file 'data/cost_noise_2016_3.csv'...ok
Writing to file 'data/cost_noise_2016_4.csv'...ok
Writing to file 'data/cost_noise_2017_1.csv'...ok
Writing to file 'data/cost_noise_2017_2.csv'...ok
Writing to file 'data/cost_noise_2017_3.csv'...ok
Writing to file 'data/cost_noise_2017_4.csv'...ok
Writing to file 'data/cost_noise_2018_1.csv'...ok
Writing to file 'data/cost_noise_2018_2.csv'...ok
Writing to file 'data/cost_noise_2018_3.csv'...ok
Writing to file 'data/cost_noise_2018_4.csv'...ok
Writing to file 'data/cost_noise_2019_1.csv'...ok
Writing to file 'data/cost_noise_2019_2.csv'...ok
Writing to file 'data/cost_noise_2019_3.csv'...ok
Writing to file 'data/cost_noise_2019_4.csv'...ok
Writing to file 'data/cost_noise_2020_1.csv'...ok
Writing to file 'data/cost_noise_2020_2.csv'...ok
Writing to file 'data/cost_noise_2020_3.csv'...ok
Writing to file 'data/cost_noise_2020_4.csv'...ok
